# Solar for Aarhus
## Can we get solar panels on every school and daycare roof in Aarhus Kommune?

In [1]:

import pandas as pd
schools = pd.read_csv("../data/raw/skoler.csv") #list of schools in Aarhus Kommune
panels = pd.read_csv("../data/raw/metadata.csv") #list of panels in Aarhus Kommune

In [2]:
panels.columns = panels.columns.str.strip()

In [3]:
#sample = pd.read_csv("../data/raw/aktuel.csv") #not used yet. Data about how much power is produced

# All Department of Children and Young People (MBU) properties with Solar Panels
Preliminary information gathering

In [4]:
panels.drop(columns=['_id', 'date'], inplace=True) #take out unneeded columns

In [5]:
panels['magistrat'] = panels['magistrat'].str.strip() #strip whitespace for later merge
panels['location'] = panels['location'].str.strip()
panels.loc[:, 'location'] = panels['location'].str.strip() #take out whitespace
panels.loc[:, 'sid'] = panels['sid'].astype(int)#change SID to integer

In [6]:
MBU_panels = panels.loc[panels['magistrat'] == 'MBU'] #filter for MBU panels

In [7]:
MBU_panels.drop(columns=['magistrat'], inplace=True)
# MBU_panels.head() # debug

/var/folders/hf/6hb2wp2j1fn707lw10cw1t4h0000gn/T/ipykernel_76847/2767539301.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MBU_panels.drop(columns=['magistrat'], inplace=True)


# Data Collection
## Which schools have solar panels already? 
Data from opendata.dk. Combining schools data with panels data.

In [8]:
schools.drop(columns=['_id', 'Periode', 'Adresse', 'Postnr', 'Skolekode', 'Antal', 'LINK', 'Ledernavn', 'Ledermail', 'Opdateret'], inplace=True)
schools['Skolenavn'] = schools['Skolenavn'].str.strip()

In [9]:
# Normalise the school names
def normalize_school_name(name):
    return name.lower().replace(' ', '').replace('skolen', 'skole').strip()


# Normalize the school names in both DataFrames
MBU_panels.loc[:, 'normalized_name'] = MBU_panels.loc[:,'location'].apply(normalize_school_name)
schools.loc[:, 'normalized_name'] = schools['Skolenavn'].apply(normalize_school_name)

/var/folders/hf/6hb2wp2j1fn707lw10cw1t4h0000gn/T/ipykernel_76847/4113839958.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MBU_panels.loc[:, 'normalized_name'] = MBU_panels.loc[:,'location'].apply(normalize_school_name)


In [19]:
schools.head() #debug

,Skolenavn,lat,lng,Postdist,normalized_name
0,Stensagerskolen,56.166771,10.137611,Brabrand,stensagerskole
1,Bakkegårdsskolen,56.253349,10.148964,Trige,bakkegårdsskole
2,Ellevangskolen,56.203037,10.217692,Risskov,ellevangskole
3,Beder Skole,56.061577,10.211610,Beder,bederskole
4,Elev Skole,56.241188,10.199622,Lystrup,elevskole


In [11]:
#MBU_panels.head() #debug

In [12]:
# Merge the DataFrames to add a column indicating the presence of solar panels
merged_df = schools.merge(MBU_panels, left_on='normalized_name', right_on='normalized_name', how='left', indicator=True)

merged_df.loc[:, 'sid'] = merged_df['sid'].fillna(0).astype(int)#change SID to integer


#print(merged_df) #debug

In [13]:

# Replace NaN values with 0 in the 'sid' column and convert it to integer
merged_df['sid'] = merged_df['sid'].astype(int)

# Add a new column 'Has Solar' based on the merge indicator
merged_df['Has Solar'] = merged_df['_merge'].apply(lambda x: 'Has solar' if x == 'both' else "Doesn't have solar")

# Drop the merge indicator column
merged_df = merged_df.drop(columns=['_merge', 'location'])

# Separate the dataframes
schools_with_solar = merged_df[merged_df['Has Solar'] == 'Has solar']
schools_without_solar = merged_df[merged_df['Has Solar'] == "Doesn't have solar"]

schools_without_solar = schools_without_solar.sort_values(by='Skolenavn') #alphabetise
schools_with_solar=schools_with_solar.sort_values(by='Skolenavn')

#Reset index for the resulting dataframes
schools_with_solar.reset_index(drop=True, inplace=True)
schools_without_solar.reset_index(drop=True, inplace=True)


In [21]:
merged_df = merged_df.drop(columns=['normalized_name'])
print(merged_df)

               Skolenavn        lat        lng     Postdist    sid  \
0        Stensagerskolen  56.166771  10.137611     Brabrand      0   
1       Bakkegårdsskolen  56.253349  10.148964        Trige  22224   
2         Ellevangskolen  56.203037  10.217692      Risskov      0   
3            Beder Skole  56.061577  10.211610        Beder      0   
4             Elev Skole  56.241188  10.199622      Lystrup      0   
5           Elsted Skole  56.241524  10.228948      Lystrup  60129   
6           Engdalskolen  56.152943  10.109608     Brabrand      0   
7     Gammelgaardsskolen  56.159332  10.157077       Åbyhøj      0   
8            Sødalskolen  56.150375  10.136786     Brabrand      0   
9            Hasle Skole  56.169296  10.163680     Aarhus V  16022   
10        Bavnehøj Skole  56.106026  10.096877   Hasselager      0   
11           Holme Skole  56.116238  10.178248     Højbjerg  16008   
12         Højvangskolen  56.129898  10.115003       Viby J  54388   
13    Katrinebjergsk

## Daycares

In [15]:
# Filter out the schools with solar panels from the MBU panels DataFrame
daycare_locations = MBU_panels[~MBU_panels['normalized_name'].isin(schools_with_solar['normalized_name'])]

daycare_locations.drop(columns=['normalized_name'], inplace=True)
# Display the resulting DataFrame
print(daycare_locations)

      sid                 location
1   54399            Ajstrupvej 49
3   60127            Ellekærskolen
6   16017         Gammelgårdskolen
10  57123       Indelukket 20 (KF)
11  54409             Indelukket 5
12  54374          Jernaldervej 3A
13  54386          Kantorvænget 33
16  57120    L.A.Rings Vej 56 (KF)
17  54376          Lindenborgvej 7
21  57117  Møllevangs alle 20 (KF)
22  54392             Nygårdsvej 5
23  54403     Pilegårdsvej 95 (BM)
32  54280     Strandvejen  38 (BM)
36  16013         Vestergårdskolen
38    298           Vidtskuevej 25


/var/folders/hf/6hb2wp2j1fn707lw10cw1t4h0000gn/T/ipykernel_76847/2101282922.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daycare_locations.drop(columns=['normalized_name'], inplace=True)


In [16]:
#print(schools_with_solar)
schools_with_solar.drop(columns=['normalized_name'], inplace=True)
schools_without_solar.drop(columns=['normalized_name'], inplace=True)


In [17]:
schools_with_solar.to_csv("../data/processed/schools_with_solar.csv", index=False)
schools_without_solar.to_csv("../data/processed/schools_without_solar.csv", index=False)


In [22]:
merged_df.to_csv("../data/processed/schools.csv", index=False)

In [18]:
MBU_panels = MBU_panels.sort_values(by='location') #alphabetise
#print(MBU_panels)

# API for querying "aktuel"
## Query example (first 5 results):

https://admin.opendata.dk/api/3/action/datastore_search?resource_id=251528ca-8ec9-4b70-9960-83c4d0c4e7b6 &limit=5

## Query example (results containing "jones"):

https://admin.opendata.dk/api/3/action/datastore_search?q=jones&resource_id=251528ca-8ec9-4b70-9960-83c4d0c4e7b6

## Query example (via SQL statement):

https://admin.opendata.dk/api/3/action/datastore_search_sql?sql=SELECT * from "251528ca-8ec9-4b70-9960-83c4d0c4e7b6" WHERE title LIKE 'jones'


# Querying API for produktion

## Query example (first 5 results):

https://admin.opendata.dk/api/3/action/datastore_search?resource_id=3028c112-a089-474d-a9c1-73d1c8352fca &limit=5

## Query example (results containing "jones"):

https://admin.opendata.dk/api/3/action/datastore_search?q=jones&resource_id=3028c112-a089-474d-a9c1-73d1c8352fca

## Query example (via SQL statement):

https://admin.opendata.dk/api/3/action/datastore_search_sql?sql=SELECT * from "3028c112-a089-474d-a9c1-73d1c8352fca" WHERE title LIKE 'jones'